In [1]:
# -*- coding: UTF-8 -*-  
import requests
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
from requests.exceptions import HTTPError
import urllib.request
import json
from urllib.request import urlopen, quote

In [1]:
def geocodeB(address):
    par = {'address': address, 'key': 'cb649a25c1f81c1451adbeca73623251'}
    base = 'http://restapi.amap.com/v3/geocode/geo'
    response = requests.get(base, par)
    answer = response.json()
    GPS=answer['geocodes'][0]['location'].split(",")
    return GPS[0],GPS[1]

In [2]:
def geocodeB(address):
    url = 'http://api.map.baidu.com/geocoding/v3/'
    output = 'json'
    ak = 'GEgyN8GVM6hqudD0pxOOPzE45j1egCjW'
    add = quote(address) #为防止乱码，先用quote进行编码
    city= quote("深圳市")
    uri = url + '?' + 'address=' + add + '&output=' + output + '&ak=' + ak+'&city='+city
    req = urlopen(uri)
    res = req.read().decode() #将其他编码的字符串解码成unicode
    temp = json.loads(res) #对json数据进行解析
    print(temp)
    a1=temp['result']['location']['lng']
    a2=temp['result']['location']['lat']
    return a1,a2

In [27]:
def geodistance(lng1,lat1,lng2,lat2):
    lng1, lat1, lng2, lat2 = map(radians, [lng1, lat1, lng2, lat2])
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    dis=2*asin(sqrt(a))*6371*1000
    return dis

In [37]:
CLocation=pd.read_excel("/Users/hanweiyu/Downloads/companyLocation.xlsx")
HLocation=pd.read_excel("/Users/hanweiyu/Downloads/houseLocation.xlsx")
SLocation=pd.read_excel("/Users/hanweiyu/Downloads/schoolLocation.xlsx")

In [38]:
CList=CLocation.values.tolist()
HList=HLocation.values.tolist()
#SList=SLocation.values.tolist()
SList=[['福新小学']]
print(CList)
print(HList)
print(SList)

[['福新小学', 113.8123, 22.67114], ['竹园小学', 114.023085784769, 22.54682684936986], ['竹林小学', 113.8700126394777, 22.80254057362558], ['珠光小学', 113.9700009596577, 22.57133754472785], ['众孚小学', 114.0557327222462, 22.52689021651086], ['中科院深圳先进技术研究院实验学校', 113.9967569892635, 22.60091597523999], ['月亮湾小学', 113.8986919411798, 22.4949428256979], ['园岭实验小学', 114.107641934937, 22.55953286009508], ['园丁学校', 113.9279684363847, 22.56118444106138], ['育才太子湾学校小学', 113.9268546273833, 22.49114442026893], ['育才四小', 113.9437887642074, 22.50061640395324], ['育才三小', 113.9261601473937, 22.50393574628426], ['育才第一小学', 113.9270498291705, 22.49704078986739], ['育才第二小学', 113.9370858779858, 22.50322589043582], ['益田小学', 114.0586031676718, 22.51772202849505], ['依山郡小学', 114.2292665721315, 22.73752003051387], ['阳光小学深圳市布吉阳光小学', 114.1206336783708, 22.6223891172804], ['阳光小学', 113.9191799542613, 22.52954234872388], ['燕山小学', 113.8613599811474, 22.80384221565023], ['盐田外国语小学', 114.2491617035766, 22.56155540572288], ['学府小学（海文部）', 113.939688

In [25]:
def judgedist(lng,lat,companyList):
    dis=100000000000
    hList=[]
    hList.append(lng)
    hList.append(lat)
    for i in companyList:
        lList=[]
        lList.append(i[1])
        lList.append(i[2])
        #print(i[0])
        newdis=geodistance(hList[0],hList[1],lList[0],lList[1])
        if newdis < dis :
            dis=newdis
    return dis

In [84]:
#print(judgedist('深圳市南山区华侨城波托菲诺纯水岸一期',CList))

2943.1687348476175


In [8]:
def getH(houselist):
    list=[]
    for i in houselist:
        sublist=geocodeB(i[0])
        list.append(sublist)
    return list

In [22]:
def getFin(houseList,companyList):
    finList=[]
    for i in houseList:
        temp=judgedist(i[1],i[2],companyList)
        finList.append(temp)
    return finList

In [39]:
def main(companyList,houseList,schoolList,path1,path2):
    l=getFin(houseList,companyList)
    #l1=getH(companyList)
    #l2=getH(houseList)
    #l3=getH(schoolList)
    df=pd.DataFrame(l)
    df.to_excel(path2)
    #df1=pd.DataFrame(l1)
    #df1.to_excel(path1)
    #df2=pd.DataFrame(l2)
    #df2.to_excel(path2)
    #df=pd.DataFrame(l3)
    #df.to_excel(path2)

In [40]:
if __name__=="__main__":
    path1='/Users/hanweiyu/Downloads/cData.xlsx'
    path2='/Users/hanweiyu/Downloads/hData.xlsx'
    main(CList,HList,SList,path1,path2)